# Multi-Objective Bayesian Optimization Tutorial

This notebook provides a comprehensive guide to understanding and using the high-performance multi-objective Bayesian optimization implementation.

## Table of Contents
1. [Introduction](#introduction)
2. [Theory Overview](#theory-overview)
3. [Implementation Features](#implementation-features)
4. [Basic Usage](#basic-usage)
5. [Advanced Configuration](#advanced-configuration)
6. [Visualization & Analysis](#visualization--analysis)
7. [Performance Tips](#performance-tips)

## Introduction

Bayesian Optimization (BO) is a powerful technique for optimizing expensive-to-evaluate functions. This implementation focuses on **multi-objective optimization** where we want to optimize multiple competing objectives simultaneously.

### Key Features:
- 🚀 **High Performance**: Numba-accelerated computations
- 🎯 **Multi-Objective**: Optimize multiple objectives simultaneously
- 📊 **Rich Visualization**: Interactive heatmaps and Pareto analysis
- 🔧 **Flexible**: Customizable hyperparameters and acquisition functions
- 📈 **Batch Optimization**: Evaluate multiple points per iteration

## Theory Overview

### Gaussian Process Regression
The core of Bayesian optimization is modeling the objective function(s) using Gaussian Processes (GPs):

$$f(x) \sim \mathcal{GP}(\mu(x), k(x, x'))$$

Where:
- $\mu(x)$ is the mean function (prior belief about function values)
- $k(x, x')$ is the kernel function (models similarity between points)

### RBF Kernel
We use the Radial Basis Function (RBF) kernel:

$$k(x, x') = \sigma^2 \exp\left(-\frac{||x - x'||^2}{2\ell^2}\right)$$

Where:
- $\sigma^2$ is the signal variance (`prior_variance`)
- $\ell$ is the length scale (`length_scales`)

### Multi-Objective Acquisition
We use Upper Confidence Bound (UCB) for each objective:

$$\text{UCB}(x) = \mu(x) + \beta \sigma(x)$$

The final acquisition function combines all objectives:

$$\text{Acquisition}(x) = \sum_{i=1}^{m} \text{UCB}_i(x)$$

## Implementation Features

### Core Components:

1. **Latin Hypercube Sampling (LHS)**: Efficient initial sampling
2. **Parallel Kernel Computation**: Fast GP operations with Numba
3. **Hyperparameter Optimization**: Automatic tuning via MLL maximization
4. **Batch Point Selection**: Evaluate multiple points per iteration
5. **Pareto Analysis**: Find optimal trade-offs between objectives

### Performance Optimizations:
- **Numba JIT compilation** for critical functions
- **Parallel kernel matrix computation**
- **Memory-efficient matrix operations**
- **Intelligent caching and reuse**

## Basic Usage

Let's start with a simple example using the built-in toy function:

In [36]:
# Import the module
import numpy as np

import time

from bayesopt import BayesianOptimization
from bayesopt.callbacks import (
    PlotterCallback,
    ProgressLogger,
    GraphSaverCallback,
    PerformanceMonitor,
    OptimizationLogger,
)
from examples.benchmark_functions import toy_function
from plotting import PyQtPlotter, StaticPlotter

np.random.seed(42)


In [37]:
# Define bounds for the search space
X_MAX = 300
Y_MAX = 300

# Note: Upper bounds are EXCLUSIVE (300 means 0-299 inclusive)
bounds = [
    (0, X_MAX),  # x dimension: 0 to 299
    (0, Y_MAX),  # y dimension: 0 to 299
]


# Initialize PyQtGraph Plotter (fast, OpenGL-accelerated)
plotter = PyQtPlotter(
    bounds=bounds,
    n_objectives=len(bounds),
)

# Setup callbacks
plotter_callback = PlotterCallback(plotter)

progress_logger = ProgressLogger(
    log_file="outputs/logs/optimization.log", verbose=True
)

performance_logger = PerformanceMonitor()

optimization_logger = OptimizationLogger()

graph_saver = GraphSaverCallback(
    plotter_class=StaticPlotter,
    bounds=bounds,
    n_objectives=len(bounds),
    save_every=1,
    save_format="png",
)

📁 Graph saver initialized. Saving to: outputs\figures\run_20260215_191820


### Create and Run the Optimizer

In [38]:
# Create optimizer with callbacks
optimizer = BayesianOptimization(
    toy_function,
    bounds,
    n_objectives=len(bounds),
    initial_samples=(X_MAX + Y_MAX) // 100,  # 1% of grid size
    n_iterations=15,
    batch_size=X_MAX // 100,  # 1% of grid size
    betas=np.array([2.0] * len(bounds)),
    callbacks=[
        # plotter_callback,
        progress_logger,
        optimization_logger,
        performance_logger,
        graph_saver,
    ],
)

In [39]:
# Run the optimization
start_time = time.time()
print("🚀 Starting optimization...\n")

optimizer.optimize()

end_time = time.time()
print(f"\n🎉 Optimization completed in {end_time - start_time:.2f} seconds.")

🚀 Starting optimization...

  └─ Iter 6: Evaluated 9 point(s)
      Point 1: x=[292.,204.] → objectives=[-20064., -2896.]
      Point 2: x=[182., 65.] → objectives=[ -924.,-7205.]
      Point 3: x=[223.,198.] → objectives=[-5229.,-2284.]
      Point 4: x=[ 93.,118.] → objectives=[-3149.,-1004.]
      Point 5: x=[107.,296.] → objectives=[ -1749.,-21296.]
      Point 6: x=[44.,33.] → objectives=[-11136.,-13669.]
      Point 7: x=[146.,154.] → objectives=[84., 4.]
      Point 8: x=[146.,155.] → objectives=[84.,-5.]
      Point 9: x=[145.,155.] → objectives=[75.,-5.]
     Best so far: obj0=84.0000 at [146.,154.], obj1=4.0000 at [146.,154.] ⭐ NEW BEST [obj0,obj1] (0.050s)

🔄 Iteration 6 (n_evaluations=9)
🔍 Selected 3 points for next batch
[Timing] Hyperparams: 0.0163s | Kernels: 0.0171s | Acquisition: 0.0169s | Eval: 0.0000s | TOTAL: 0.0503s
  💾 Saved plot: iteration_0006.png
  └─ Iter 9: Evaluated 3 point(s)
      Point 10: x=[  0.,186.] → objectives=[-22400., -1276.]
      Point 11: x=[  

### Analyze Results

In [40]:
# Create animated GIF from saved images
graph_saver.finalize()

# Analyze results
optimizer.pareto_analysis()

# Log performance summary
performance_logger.summary()

🎬 Creating GIF from 15 images...
✅ GIF created: outputs\figures\run_20260215_191820\optimization.gif
   Frames: 15 | Duration: 500ms per frame
📊 Pareto Analysis Results:
Input: [150. 150.], Pareto Point 1: [100.  20.]

📊 Performance Summary:
  Total iterations: 15
  Average time per iteration: 0.060s
  Total time: 0.91s

  Breakdown (average):
    Hyperparams: 0.0123s (20.4%)
    Kernels: 0.0187s (30.9%)
    Acquisition: 0.0294s (48.7%)
    Evaluation: 0.0000s (0.0%)


## Visualization & Analysis

### Understanding the Heatmap Plots

When you run the 2D optimization, you'll see heatmap plots with three panels per objective:

1. **Mean Prediction (μ)**: Shows the GP's prediction of objective values
2. **Uncertainty (σ)**: Shows where the GP is uncertain (high variance)
3. **Acquisition Function**: Shows where the optimizer will sample next

### Plot Interpretation:
- **Dark blue areas** in mean plots indicate low predicted values
- **Bright yellow areas** in mean plots indicate high predicted values
- **Red areas** in uncertainty plots show high uncertainty
- **White circles** show evaluated points
- **Red stars** show next points to evaluate

## Performance Tips

### Choosing Parameters

#### Initial Samples
- **Rule of thumb**: 5-10 samples per dimension
- **More samples**: Better initial model, slower startup
- **Fewer samples**: Faster startup, more exploration needed

#### Batch Size
- **Larger batches**: Better parallelization, less adaptive
- **Smaller batches**: More adaptive, sequential evaluation
- **Sweet spot**: 2-5 points per batch

#### Beta Values (Exploration)
- **Higher β**: More exploration, slower convergence
- **Lower β**: More exploitation, faster convergence
- **Typical range**: 1.0 - 3.0

### Memory and Computation Scaling

The algorithm's complexity scales as:
- **Memory**: O(n² · m) where n = evaluations, m = objectives
- **Computation**: O(n³ · m) due to matrix inversion

**Practical limits**:
- Up to ~100-200 evaluations: Excellent performance
- 200-500 evaluations: Good performance
- 500+ evaluations: Consider reducing `n_iterations` or `batch_size`

## Summary

This tutorial covered:

✅ **Theory**: Gaussian Processes and multi-objective optimization  
✅ **Implementation**: High-performance Numba-accelerated code  
✅ **Usage**: From basic to advanced configurations  
✅ **Visualization**: Understanding heatmaps and Pareto analysis  
✅ **Performance**: Tuning parameters for optimal results  

### Key Takeaways:
1. **Start simple**: Use default parameters first
2. **Understand trade-offs**: Balance exploration vs exploitation
3. **Monitor performance**: Watch convergence and Pareto fronts
4. **Scale appropriately**: Consider memory limits for large problems

Happy optimizing! 🚀